In [38]:
from data_pipeline import *
import time
import numpy as np
from ghcn_helper import *


## Check correctness

In [28]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=150)
yearbook = build_yearbook("20180101", "20181231", ["TMAX", "TMIN"])
df = yearbook[2018]
df


latitude  longitude  TMAX  TMIN
date     station_id                                  
20180101 AE000041196   25.3330    55.5170  25.9  11.2
         AJ000037575   41.5500    46.6670   7.4   5.3
         AJ000037675   41.3670    48.5170   4.5   0.3
         AJ000037735   40.7167    46.4167  14.3   5.4
         AJ000037756   40.5330    48.9330  10.0   0.6
...                        ...        ...   ...   ...
20181231 WA007401540  -22.5670    17.1000  29.8  15.4
         WA007848390  -22.4830    17.4670  28.2  15.2
         WA012084750  -17.9170    19.7670  25.0  19.0
         WF000917530  -13.2330  -176.1670  28.8  25.8
         ZI000067775  -17.9170    31.1330  28.5  16.6

[4234648 rows x 4 columns]

In [29]:
df_day_one = df.loc["20180101"]
df_day_one = df_day_one[["TMAX", "TMIN"]]
df_day_one


,TMAX,TMIN
station_id,,
AE000041196,25.9,11.2
AJ000037575,7.4,5.3
AJ000037675,4.5,0.3
AJ000037735,14.3,5.4
AJ000037756,10.0,0.6
...,...,...
VQW00011640,30.0,23.9
WA007848390,34.1,11.6
WA007878380,37.0,14.8


In [30]:
msk_lap, x, non_zero_idx = ghcn[0]
blacklist = ghcn.blacklist
df_day_one = df_day_one.drop(blacklist, axis=0, errors="ignore")

non_zero_x = x[non_zero_idx]
print(non_zero_x)

assert np.allclose(df_day_one.values, non_zero_x.numpy())


---previous Laplacian expired or no previous Laplacian; building new---
Found 13778 stations in total in date range 20180101-20180530
Successfully converged after 105 steps
Difference between ML @ v and lambda * v:  6.447025501311058
tensor([[25.9000, 11.2000],
        [ 7.4000,  5.3000],
        [ 4.5000,  0.3000],
        ...,
        [37.0000, 14.8000],
        [26.7000, 12.2000],
        [30.9000, 24.3000]], dtype=torch.float64)


## Showcase dataset usage

In [21]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=150)


In [15]:
for i in range(365):
    print("Iter: ", i)
    t = time.time()
    msk_lap, x, non_zero_idx = ghcn[i]
    print("Time Taken: ", time.time() - t)
    


Iter:  0
---previous Laplacian expired or no previous Laplacian; building new---
Found 13778 stations in total in date range 20180101-20180530
Successfully converged after 107 steps
Difference between ML @ v and lambda * v:  5.726393775964241
Time Taken:  20.024678945541382
num-non-zero:  11406
num non zero in x:  22599
Iter:  1
Time Taken:  0.21387624740600586
num-non-zero:  11523
num non zero in x:  22799
Iter:  2
Time Taken:  0.29879307746887207
num-non-zero:  11421
num non zero in x:  22595
Iter:  3
Time Taken:  0.20823216438293457
num-non-zero:  11648
num non zero in x:  23019
Iter:  4
Time Taken:  0.20981192588806152
num-non-zero:  11547
num non zero in x:  22855
Iter:  5
Time Taken:  0.19684100151062012
num-non-zero:  11481
num non zero in x:  22679
Iter:  6
Time Taken:  0.29671192169189453
num-non-zero:  11355
num non zero in x:  22363
Iter:  7
Time Taken:  0.19531917572021484
num-non-zero:  11515
num non zero in x:  22602
Iter:  8
Time Taken:  0.21743202209472656
num-non-zero:

Time Taken:  0.3077418804168701
num-non-zero:  11543
num non zero in x:  22776
Iter:  91
Time Taken:  0.19471192359924316
num-non-zero:  11618
num non zero in x:  22904
Iter:  92
Time Taken:  0.2017512321472168
num-non-zero:  11662
num non zero in x:  23051
Iter:  93
Time Taken:  0.30288100242614746
num-non-zero:  11488
num non zero in x:  22658
Iter:  94
Time Taken:  0.21684908866882324
num-non-zero:  11755
num non zero in x:  23204
Iter:  95
Time Taken:  0.20684003829956055
num-non-zero:  11739
num non zero in x:  23209
Iter:  96
Time Taken:  0.19763684272766113
num-non-zero:  11556
num non zero in x:  22800
Iter:  97
Time Taken:  0.3032820224761963
num-non-zero:  11536
num non zero in x:  22702
Iter:  98
Time Taken:  0.21451091766357422
num-non-zero:  11690
num non zero in x:  22964
Iter:  99
Time Taken:  0.2177889347076416
num-non-zero:  11655
num non zero in x:  22994
Iter:  100
Time Taken:  0.2983827590942383
num-non-zero:  11611
num non zero in x:  22922
Iter:  101
Time Taken:  

Time Taken:  0.25760602951049805
num-non-zero:  11704
num non zero in x:  23396
Iter:  179
Time Taken:  0.2641720771789551
num-non-zero:  11610
num non zero in x:  23210
Iter:  180
Time Taken:  0.22873187065124512
num-non-zero:  11538
num non zero in x:  23060
Iter:  181
Time Taken:  0.38579893112182617
num-non-zero:  11542
num non zero in x:  23060
Iter:  182
Time Taken:  0.2552168369293213
num-non-zero:  11892
num non zero in x:  23774
Iter:  183
Time Taken:  0.253324031829834
num-non-zero:  11597
num non zero in x:  23166
Iter:  184
Time Taken:  0.3441338539123535
num-non-zero:  11865
num non zero in x:  23707
Iter:  185
Time Taken:  0.28955602645874023
num-non-zero:  11936
num non zero in x:  23863
Iter:  186
Time Taken:  0.25736021995544434
num-non-zero:  11700
num non zero in x:  23394
Iter:  187
Time Taken:  0.26910901069641113
num-non-zero:  11531
num non zero in x:  23054
Iter:  188
Time Taken:  0.32085371017456055
num-non-zero:  11020
num non zero in x:  22033
Iter:  189
Time

Time Taken:  0.2245340347290039
num-non-zero:  11527
num non zero in x:  22920
Iter:  270
Time Taken:  0.21863007545471191
num-non-zero:  11512
num non zero in x:  22905
Iter:  271
Time Taken:  0.21652603149414062
num-non-zero:  11361
num non zero in x:  22570
Iter:  272
Time Taken:  0.32254600524902344
num-non-zero:  11446
num non zero in x:  22732
Iter:  273
Time Taken:  0.26157283782958984
num-non-zero:  11478
num non zero in x:  22835
Iter:  274
Time Taken:  0.28164076805114746
num-non-zero:  11558
num non zero in x:  23034
Iter:  275
Time Taken:  0.3747262954711914
num-non-zero:  11547
num non zero in x:  22997
Iter:  276
Time Taken:  0.22752904891967773
num-non-zero:  11556
num non zero in x:  22972
Iter:  277
Time Taken:  0.21648788452148438
num-non-zero:  11516
num non zero in x:  22792
Iter:  278
Time Taken:  0.2196969985961914
num-non-zero:  11399
num non zero in x:  22580
Iter:  279
Time Taken:  0.3227362632751465
num-non-zero:  11458
num non zero in x:  22731
Iter:  280
Tim

Time Taken:  0.20807504653930664
num-non-zero:  10980
num non zero in x:  21522
Iter:  357
Time Taken:  0.20636224746704102
num-non-zero:  10921
num non zero in x:  21420
Iter:  358
Time Taken:  0.292813777923584
num-non-zero:  10937
num non zero in x:  21487
Iter:  359
Time Taken:  0.19927597045898438
num-non-zero:  11168
num non zero in x:  21971
Iter:  360
Time Taken:  0.21003079414367676
num-non-zero:  11153
num non zero in x:  21907
Iter:  361
Time Taken:  0.32653117179870605
num-non-zero:  11218
num non zero in x:  22080
Iter:  362
Time Taken:  0.22005319595336914
num-non-zero:  11099
num non zero in x:  21807
Iter:  363
Time Taken:  0.21227788925170898
num-non-zero:  11178
num non zero in x:  22005
Iter:  364
Time Taken:  0.21384096145629883
num-non-zero:  11126
num non zero in x:  21936


In [39]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=3)
sampler = Sampler(times=3, size=0.01)
ghcn_impainter = GHCNImpainter(ghcn, sampler)


In [40]:
for i in range(12):
    print("ITER: ", i)
    further_masked_laplacian, masked_x, idx_to_mask, masked_value = ghcn_impainter[i]


ITER:  0
---previous Laplacian expired or no previous Laplacian; building new---
Found 12533 stations in total in date range 20180101-20180103
Successfully converged after 100 steps
Difference between ML @ v and lambda * v:  6.0857132129298135


AttributeError: 'Tensor' object has no attribute 'copy'